# Convertitore Dicom To Mp4

Librerie necessarie al corretto funzionamento degli scripts:
1. [Numpy](https://pypi.org/project/numpy/)
1. [Pydicom](https://pypi.org/project/pydicom/)
1. [Os](https://pypi.org/project/OS-Platform/)
1. [Cv2](https://pypi.org/project/cv_utils/)

Prima di procedere all'importazione delle stesse è necessario procedere con l'installazione!

In [1]:
# Per OS windows --> Jupyter-notebook
!conda install numpy
!conda install pydicom
!conda install os
!conda install cv2

# Per OS linux
!pip install numpy
!pip install pydicom
!pip install os
!pip install cv2


CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://repo.anaconda.com/pkgs/main/win-64/current_repodata.json>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.

If your current network has https://www.anaconda.com blocked, please file
a support request with your network engineering team.

'https//repo.anaconda.com/pkgs/main/win-64'





CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://repo.anaconda.com/pkgs/main/win-64/current_repodata.json>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.

If your current network has https://www.anaconda.com blocked, please file
a support request with your network engineering team.

'https//repo.anaconda.com/pkgs/main/win-64'



CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://repo.anaconda.com/pkgs/main/win-64/current_repodata.json>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.

If your current network has https://www.anaconda.com blocked, please file
a support request with your network engineering team.

'https//repo.anaconda.com/pkgs/main/win-64'





CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://repo.anaconda.com/pkgs/main/win-64/current_repodata.json>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.

If your current network has https://www.anaconda.com blocked, please file
a support request with your network engineering team.

'https//repo.anaconda.com/pkgs/main/win-64'




ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [6]:
import numpy as np
import pydicom
import os
import cv2

ModuleNotFoundError: No module named 'pydicom'

In [ ]:
from glob import glob
import pydicom
import os
import cv2
from PIL import Image
import re
import numpy as np

# Load the DICOMDIR file
dicomdir_path = 'dcm_files/DICOMDIR'
dicomdir = pydicom.dcmread(dicomdir_path)

record = None
for record in dicomdir.DirectoryRecordSequence:
    if record.DirectoryRecordType =='IMAGE' and record.InstanceNumber == 1:
        break

filename = os.path.join(os.path.dirname('dcm_files/DICOMDIR'), record.ReferencedFileID[1])
ds = pydicom.dcmread(filename)
# print(ds)
num_frames = ds.NumberOfFrames

pixel_data = ds.pixel_array
pixel_data = cv2.normalize(pixel_data, None, 0, 255, cv2.NORM_MINMAX,cv2.CV_8U)
print(pixel_data.shape)

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

# extract images from the dcm
for i in range(pixel_data.shape[0]):
    os.makedirs('Output/dicom_3d_images', exist_ok=True)
    slice = pixel_data[i, :, :, :]
    if ds.PhotometricInterpretation == 'YBR_FULL_422':
        slice = cv2.cvtColor(slice, cv2.COLOR_YUV2RGB)
    img = Image.fromarray(slice)
    img.save(f'Output/dicom_3d_images/image{i}.jpg')

# write images into a video
file_path = glob('Output/dicom_3d_images/*.jpg')
height, width, _ = cv2.imread(file_path[0]).shape
size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = ds.CineRate
out = cv2.VideoWriter('Output/image1.mp4', fourcc, fps, size)

for filename in sorted(file_path, key=numericalSort):
    img = cv2.imread(filename)
    out.write(img)
out.release()